In [15]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os


In [16]:
import dask
import dask.dataframe as dd

In [17]:
meta_sample = pd.read_csv('metadata_sample_submission.csv',sep=',') 
data_sample = pd.read_csv('sample_submission.csv',sep=',') 


In [18]:
data_sample.head()

,index,percentage_docks_available
0,0,0.029106
1,1,0.585888
2,2,0.015558
3,3,0.267947
4,4,0.098821


In [19]:
data_sample = pd.merge(meta_sample, data_sample, how='left')


In [20]:
data_sample.head()

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,percentage_docks_available
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074,0.029106
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449,0.585888
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852,0.015558
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111,0.267947
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160,0.098821


In [21]:
#print the sample dataset size
print("There is", data_sample.shape[0], "samples")
print("Each sample has", data_sample.shape[1], "features")
data_sample.head()

There is 54999 samples
Each sample has 10 features


,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,percentage_docks_available
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074,0.029106
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449,0.585888
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852,0.015558
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111,0.267947
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160,0.098821


In [22]:
#daskfile = dd.read_cvs("")

AttributeError: module 'dask.dataframe' has no attribute 'read_cvs'

In [34]:
#this function of reading files and making them dask dataframes only works for a certain filename format that is data_YEAR_MONTH like data_21_12 
def merge_datasets_of_year(year, starting_month):
    merged_df = pd.DataFrame()
    for month in range(starting_month, 13):
        if month < 10:
            month = "0"+str(month)
        file_name = str(year)+"_"+str(month)+".csv"
        df_name = "data_"+str(year)+"_"+str(month)
        #dask dataframe syntax here: 
        new_df = dd.read_csv(file_name,sep=',', dtype={'last_reported': 'float64'})
        merged_df = dd.concat([merged_df, new_df]).reset_index(drop=True)
    
    return merged_df

In [35]:
data_2019 = merge_datasets_of_year(2019, 3)


In [36]:
data_2020 = merge_datasets_of_year(2020, 1)


In [37]:
data_2021 = merge_datasets_of_year(2021, 1)


In [38]:
data_2022 = merge_datasets_of_year(2021, 1)


In [39]:
data_2019['last_reported'] = dd.to_datetime(data_2019['last_reported'], unit='s')
data_2020['last_reported'] = dd.to_datetime(data_2020['last_reported'], unit='s')
data_2021['last_reported'] = dd.to_datetime(data_2021['last_reported'], unit='s')
data_2022['last_reported'] = dd.to_datetime(data_2022['last_reported'], unit='s')


In [40]:
data_2022.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl,traffic
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,1609455626,22,NaN
1,2,17,15,2,10,2020-12-31 22:57:51,True,IN_SERVICE,1,1,1,1609455626,22,NaN
2,3,3,3,0,22,2020-12-31 22:56:48,True,IN_SERVICE,1,1,1,1609455626,22,NaN
3,4,15,14,1,6,2020-12-31 22:57:38,True,IN_SERVICE,1,1,1,1609455626,22,NaN
4,5,8,8,0,31,2020-12-31 22:57:21,True,IN_SERVICE,1,1,1,1609455626,22,NaN


In [41]:
data_2022['hour'] = data_2022['last_reported'].dt.hour
data_2022['minute'] = data_2022['last_reported'].dt.minute
data_2022['weekday'] = data_2022['last_reported'].dt.weekday
data_2022['month'] = data_2022['last_reported'].dt.month


In [42]:
data_2022['day'] = data_2022['last_reported'].dt.day

In [43]:
data_2022['availability'] = data_2022['num_docks_available'] / (data_2022['num_bikes_available']+data_2022['num_docks_available']) * 100


In [44]:
data_2022.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl,traffic,hour,minute,weekday,month,day,availability
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,1609455626,22,NaN,22,56,3,12,31,6.521739
1,2,17,15,2,10,2020-12-31 22:57:51,True,IN_SERVICE,1,1,1,1609455626,22,NaN,22,57,3,12,31,37.037037
2,3,3,3,0,22,2020-12-31 22:56:48,True,IN_SERVICE,1,1,1,1609455626,22,NaN,22,56,3,12,31,88.000000
3,4,15,14,1,6,2020-12-31 22:57:38,True,IN_SERVICE,1,1,1,1609455626,22,NaN,22,57,3,12,31,28.571429
4,5,8,8,0,31,2020-12-31 22:57:21,True,IN_SERVICE,1,1,1,1609455626,22,NaN,22,57,3,12,31,79.487179


In [45]:
data_2022 = data_2022[data_2022["status"]=="IN_SERVICE"]

In [46]:
data_2022 = data_2022[data_2022["is_renting"]==1]

In [47]:
data_2022 = data_2022[data_2022["is_returning"]==1]

In [60]:
data_2022.groupby(["station_id", "month", "day", "hour"])["availability"].mean().reset_index().compute()

,station_id,month,day,hour,availability
0,1,1,1,0,11.956522
1,1,1,1,1,10.869565
2,1,1,1,2,10.869565
3,1,1,1,3,10.869565
4,1,1,1,4,10.869565
...,...,...,...,...,...
4404184,519,12,31,17,94.791667
4404185,519,12,31,18,90.972222
4404186,519,12,31,19,85.763889
4404187,519,12,31,20,84.722222
